# Hash vs JSON Storage

Out of the box, Redis provides a [variety of data structures](https://redis.com/redis-enterprise/data-structures/) that can adapt to your domain specific applications and use cases.
In this notebook, we will demonstrate how to use RedisVL4J with both [Hash](https://redis.io/docs/data-types/hashes/) and [JSON](https://redis.io/docs/data-types/json/) data.

Before running this notebook, be sure to:
1. Have installed RedisVL4J and have that environment active for this notebook.
2. Have a running Redis Stack or Redis Enterprise instance with RediSearch > 2.4 activated.

For example, you can run [Redis Stack](https://redis.io/docs/install/install-stack/) locally with Docker:

```bash
docker run -d -p 6379:6379 -p 8001:8001 redis/redis-stack:latest
```

Or create a [FREE Redis Cloud](https://redis.io/cloud).

In [1]:
// Load Maven dependencies
%maven redis.clients:jedis:5.2.0
%maven org.slf4j:slf4j-nop:2.0.16
%maven com.fasterxml.jackson.core:jackson-databind:2.18.0
%maven com.fasterxml.jackson.dataformat:jackson-dataformat-yaml:2.18.0
%maven net.razorvine:pickle:1.4
%maven de.vandermeer:asciitable:0.3.2
%maven com.github.f4b6a3:ulid-creator:5.2.3

// Add LangChain4J dependencies for embeddings
%maven dev.langchain4j:langchain4j-embeddings:0.36.2
%maven dev.langchain4j:langchain4j-embeddings-all-minilm-l6-v2:0.36.2

// Import RedisVL4J classes
import com.redis.vl.index.SearchIndex;
import com.redis.vl.schema.IndexSchema;
import com.redis.vl.query.VectorQuery;
import com.redis.vl.query.Filter;

// Import Redis client
import redis.clients.jedis.UnifiedJedis;
import redis.clients.jedis.HostAndPort;
import redis.clients.jedis.search.SearchResult;

// Import pickle library
import net.razorvine.pickle.Unpickler;

// Import ASCII Table
import de.vandermeer.asciitable.AsciiTable;
import de.vandermeer.asciitable.CWC_LongestLine;

// Import ULID generator
import com.github.f4b6a3.ulid.UlidCreator;

// Import LangChain4J for embeddings
import dev.langchain4j.data.embedding.Embedding;
import dev.langchain4j.model.embedding.EmbeddingModel;
import dev.langchain4j.model.embedding.onnx.allminilml6v2.AllMiniLmL6V2EmbeddingModel;

// Import Java standard libraries
import java.util.*;
import java.nio.*;
import java.io.*;

System.out.println("Libraries imported successfully!");

Libraries imported successfully!


In [2]:
// Initialize the embedding model
// Using AllMiniLmL6V2 which is similar to the Python's all-mpnet-base-v2
// Both are sentence transformer models suitable for semantic similarity
EmbeddingModel embeddingModel = new AllMiniLmL6V2EmbeddingModel();

System.out.println("Embedding model initialized: AllMiniLmL6V2");
System.out.println("This model generates 384-dimensional embeddings");

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


Embedding model initialized: AllMiniLmL6V2
This model generates 384-dimensional embeddings


In [3]:
// Load data from pickle file
Unpickler unpickler = new Unpickler();
List<Map<String, Object>> data;

try {
    FileInputStream fileInputStream = new FileInputStream("resources/hybrid_example_data.pkl");
    data = (List<Map<String, Object>>) unpickler.load(fileInputStream);
    fileInputStream.close();

    System.out.println("Sample data loaded from pickle file with " + data.size() + " users");

} catch (Exception e) {
    System.err.println("Error loading pickle file: " + e.getMessage());
    e.printStackTrace();
    throw e;
}

// Helper method to print query results in table format
public static void resultPrint(List<Map<String, Object>> results) {
    if (results == null || results.isEmpty()) {
        System.out.println("No results to display");
        return;
    }

    // Fields to exclude from display
    Set<String> toRemove = Set.of("id", "payload", "score", "user_embedding", "__user_embedding_score");

    // Get all unique keys from results, excluding the ones we want to remove
    Set<String> allKeys = new LinkedHashSet<>();
    for (Map<String, Object> result : results) {
        for (String key : result.keySet()) {
            if (!toRemove.contains(key) && !key.contains("_distance")) {
                allKeys.add(key);
            }
        }
    }

    // Create ASCII table
    AsciiTable table = new AsciiTable();
    table.addRule();

    // Add header row
    table.addRow(allKeys.toArray());
    table.addRule();

    // Add data rows
    for (Map<String, Object> result : results) {
        List<Object> row = new ArrayList<>();
        for (String key : allKeys) {
            Object value = result.get(key);
            row.add(value != null ? value.toString() : "");
        }
        table.addRow(row.toArray());
    }
    table.addRule();

    // Configure column widths
    CWC_LongestLine cwc = new CWC_LongestLine();
    table.getRenderer().setCWC(cwc);

    // Print the table
    System.out.println(table.render());
}

// Display the loaded data
AsciiTable dataTable = new AsciiTable();
dataTable.addRule();
dataTable.addRow("User", "Age", "Job", "Credit Score", "Office Location", "User Embedding", "Last Updated");
dataTable.addRule();

data.forEach(user -> {
    Object embedding = user.get("user_embedding");
    String embStr = "<bytes>";
    if (embedding instanceof byte[]) {
        embStr = "<" + ((byte[])embedding).length + " bytes>";
    }
    dataTable.addRow(
        user.get("user"),
        user.get("age"),
        user.get("job"),
        user.get("credit_score"),
        user.get("office_location"),
        embStr,
        user.get("last_updated")
    );
});
dataTable.addRule();

CWC_LongestLine cwc = new CWC_LongestLine();
dataTable.getRenderer().setCWC(cwc);

System.out.println("\nLoaded data:");
System.out.println(dataTable.render());

Sample data loaded from pickle file with 7 users

Loaded data:
┌───────┬───┬─────────────┬────────────┬─────────────────┬──────────────┬────────────┐
│User   │Age│Job          │Credit Score│Office Location  │User Embedding│Last Updated│
├───────┼───┼─────────────┼────────────┼─────────────────┼──────────────┼────────────┤
│john   │18 │engineer     │high        │-122.4194,37.7749│<12 bytes>    │1741627789  │
│derrick│14 │doctor       │low         │-122.4194,37.7749│<12 bytes>    │1741627789  │
│nancy  │94 │doctor       │high        │-122.4194,37.7749│<12 bytes>    │1710696589  │
│tyler  │100│engineer     │high        │-122.0839,37.3861│<12 bytes>    │1742232589  │
│tim    │12 │dermatologist│high        │-122.0839,37.3861│<12 bytes>    │1739644189  │
│taimur │15 │CEO          │low         │-122.0839,37.3861│<12 bytes>    │1742232589  │
│joe    │35 │dentist      │medium      │-122.0839,37.3861│<12 bytes>    │1742232589  │
└───────┴───┴─────────────┴────────────┴─────────────────┴─────────

## Hash or JSON -- how to choose?
Both storage options offer a variety of features and tradeoffs. Below we will work through a dummy dataset to learn when and how to use both.

### Working with Hashes
Hashes in Redis are simple collections of field-value pairs. Think of it like a mutable single-level dictionary contains multiple "rows":

```java
{
    "model": "Deimos",
    "brand": "Ergonom",
    "type": "Enduro bikes",
    "price": 4972,
}
```

Hashes are best suited for use cases with the following characteristics:
- Performance (speed) and storage space (memory consumption) are top concerns
- Data can be easily normalized and modeled as a single-level dict

> Hashes are typically the default recommendation.

In [4]:
// Define the hash index schema
Map<String, Object> hashSchema = Map.of(
    "index", Map.of(
        "name", "user-hash",
        "prefix", "user-hash-docs",
        "storage_type", "hash"  // default setting -- HASH
    ),
    "fields", List.of(
        Map.of("name", "user", "type", "tag"),
        Map.of("name", "credit_score", "type", "tag"),
        Map.of("name", "job", "type", "text"),
        Map.of("name", "age", "type", "numeric"),
        Map.of("name", "office_location", "type", "geo"),
        Map.of(
            "name", "user_embedding",
            "type", "vector",
            "attrs", Map.of(
                "dims", 3,
                "distance_metric", "cosine",
                "algorithm", "flat",
                "datatype", "float32"
            )
        )
    )
);

System.out.println("Hash schema defined");

Hash schema defined


In [5]:
// Connect to Redis and construct a search index from the hash schema
UnifiedJedis jedis = new UnifiedJedis(new HostAndPort("redis-stack", 6379));

SearchIndex hindex = SearchIndex.fromDict(hashSchema, jedis);

// Create the index (no data yet)
hindex.create(true);

System.out.println("Hash index created: " + hindex.getName());

Hash index created: user-hash


In [6]:
// Show the underlying storage type
System.out.println("Storage type: " + hindex.getStorageType());

Storage type: HASH


#### Vectors as byte strings
One nuance when working with Hashes in Redis, is that all vectorized data must be passed as a byte string (for efficient storage, indexing, and processing). An example of that can be seen below:

In [7]:
// Show a single entry from the data that will be loaded
Map<String, Object> sampleUser = data.get(0);
System.out.println("Sample user entry:");
sampleUser.forEach((key, value) -> {
    if ("user_embedding".equals(key) && value instanceof byte[]) {
        System.out.println("  " + key + ": <" + ((byte[])value).length + " bytes>");
    } else {
        System.out.println("  " + key + ": " + value);
    }
});

Sample user entry:
  credit_score: high
  last_updated: 1741627789
  user_embedding: <12 bytes>
  office_location: -122.4194,37.7749
  job: engineer
  user: john
  age: 18


In [8]:
// Load hash data (vectors remain as byte arrays)
List<String> keys = hindex.load(data, "user");

System.out.println("Loaded " + keys.size() + " documents to hash index");
System.out.println("Keys created: ");
keys.forEach(key -> System.out.println("  " + key));

Loaded 7 documents to hash index
Keys created: 
  user-hash-docs:john
  user-hash-docs:derrick
  user-hash-docs:nancy
  user-hash-docs:tyler
  user-hash-docs:tim
  user-hash-docs:taimur
  user-hash-docs:joe


#### Performing Queries
Once our index is created and data is loaded into the right format, we can run queries against the index with RedisVL4J:

In [9]:
// Create a combined filter expression
Filter creditFilter = Filter.tag("credit_score", "high");
Filter jobFilter = Filter.prefix("job", "enginee");
Filter ageFilter = Filter.numeric("age").gt(17);

Filter combinedFilter = Filter.and(creditFilter, jobFilter, ageFilter);

// Create vector query with filter
VectorQuery v = VectorQuery.builder()
    .vector(new float[]{0.1f, 0.1f, 0.5f})
    .field("user_embedding")
    .returnFields("user", "credit_score", "age", "job", "office_location")
    .withPreFilter(combinedFilter.build())
    .build();

List<Map<String, Object>> results = hindex.query(v);
resultPrint(results);

DEBUG: Executing search query: '((@credit_score:{high} @job:enginee* @age:[(17 +inf]))=>[KNN $K @user_embedding $vec AS vector_distance]'
DEBUG: Search params: {vec=[B@53f7d99a, K=10}
┌────────────┬────────────┬─────────────────┬────────┬─────┬───┐
│credit_score│last_updated│office_location  │job     │user │age│
├────────────┼────────────┼─────────────────┼────────┼─────┼───┤
│high        │1741627789  │-122.4194,37.7749│engineer│john │18 │
│high        │1742232589  │-122.0839,37.3861│engineer│tyler│100│
└────────────┴────────────┴─────────────────┴────────┴─────┴───┘


In [10]:
// Clean up hash index
hindex.delete(true);
System.out.println("Hash index deleted");

Hash index deleted


### Working with JSON

JSON is best suited for use cases with the following characteristics:
- Ease of use and data model flexibility are top concerns
- Application data is already native JSON
- Replacing another document storage/db solution

In [11]:
// Define the json index schema
// The library should automatically add $.field_name paths for JSON storage
Map<String, Object> jsonSchema = Map.of(
    "index", Map.of(
        "name", "user-json",
        "prefix", "user-json-docs",
        "storage_type", "json"  // JSON storage type
    ),
    "fields", List.of(
        Map.of("name", "user", "type", "tag"),
        Map.of("name", "credit_score", "type", "tag"),
        Map.of("name", "job", "type", "text"),
        Map.of("name", "age", "type", "numeric"),
        Map.of("name", "office_location", "type", "geo"),
        Map.of(
            "name", "user_embedding",
            "type", "vector",
            "attrs", Map.of(
                "dims", 3,
                "distance_metric", "cosine",
                "algorithm", "flat",
                "datatype", "float32"
            )
        )
    )
);

System.out.println("JSON schema defined");

JSON schema defined


In [12]:
// Construct a search index from the json schema
SearchIndex jindex = SearchIndex.fromDict(jsonSchema, jedis);

// Create the index (no data yet)
jindex.create(true);

System.out.println("JSON index created: " + jindex.getName());

JSON index created: user-json


#### Vectors as float arrays
Vectorized data stored in JSON must be stored as a pure array (Java List) of floats. We will modify our sample data to account for this below:

In [13]:
// Convert byte array vectors to float arrays for JSON storage
List<Map<String, Object>> jsonData = new ArrayList<>();

for (Map<String, Object> user : data) {
    Map<String, Object> jsonUser = new HashMap<>(user);

    // Convert byte array embedding to float array for JSON
    Object embedding = user.get("user_embedding");
    if (embedding instanceof byte[]) {
        byte[] embBytes = (byte[]) embedding;
        ByteBuffer buffer = ByteBuffer.wrap(embBytes).order(ByteOrder.LITTLE_ENDIAN);
        float[] floats = new float[3];
        for (int i = 0; i < 3; i++) {
            floats[i] = buffer.getFloat();
        }
        jsonUser.put("user_embedding", floats);
    }

    jsonData.add(jsonUser);
}

System.out.println("Converted data for JSON storage");

Converted data for JSON storage


In [14]:
// Inspect a single JSON record
Map<String, Object> sampleJsonUser = jsonData.get(0);
System.out.println("Sample JSON user entry:");
sampleJsonUser.forEach((key, value) -> {
    if ("user_embedding".equals(key) && value instanceof float[]) {
        float[] vec = (float[]) value;
        System.out.println("  " + key + ": [" + vec[0] + ", " + vec[1] + ", " + vec[2] + "]");
    } else {
        System.out.println("  " + key + ": " + value);
    }
});

Sample JSON user entry:
  credit_score: high
  last_updated: 1741627789
  user_embedding: [0.1, 0.1, 0.5]
  office_location: -122.4194,37.7749
  job: engineer
  user: john
  age: 18


In [15]:
// Load JSON data
List<String> jsonKeys = jindex.load(jsonData, "user");

System.out.println("Loaded " + jsonKeys.size() + " documents to JSON index");
System.out.println("Keys created: ");
jsonKeys.forEach(key -> System.out.println("  " + key));

Loaded 7 documents to JSON index
Keys created: 
  user-json-docs:john
  user-json-docs:derrick
  user-json-docs:nancy
  user-json-docs:tyler
  user-json-docs:tim
  user-json-docs:taimur
  user-json-docs:joe


In [16]:
// For JSON storage, we need to use JSON paths in filters
Filter jsonCreditFilter = Filter.tag("$.credit_score", "high");
Filter jsonJobFilter = Filter.prefix("$.job", "enginee");
Filter jsonAgeFilter = Filter.numeric("$.age").gt(17);

Filter jsonCombinedFilter = Filter.and(jsonCreditFilter, jsonJobFilter, jsonAgeFilter);

// Create vector query with JSON paths
VectorQuery jsonV = VectorQuery.builder()
    .vector(new float[]{0.1f, 0.1f, 0.5f})
    .field("$.user_embedding")
    .returnFields("user", "credit_score", "age", "job", "office_location")
    .withPreFilter(jsonCombinedFilter.build())
    .build();

List<Map<String, Object>> jsonResults = jindex.query(jsonV);
resultPrint(jsonResults);

DEBUG: Executing search query: '((@\$\.credit_score:{high} @\$\.job:enginee* @\$\.age:[(17 +inf]))=>[KNN $K @\$\.user_embedding $vec AS vector_distance]'
DEBUG: Search params: {vec=[B@2228e74c, K=10}
┌────────┬─────┬─────────────────┬──────────────┬──────┬────────────────┬──────────────┐
│$.job   │$.age│$.office_location│$.last_updated│$.user│$.user_embedding│$.credit_score│
├────────┼─────┼─────────────────┼──────────────┼──────┼────────────────┼──────────────┤
│engineer│18   │-122.4194,37.7749│1741627789    │john  │[0.1, 0.1, 0.5] │high          │
│engineer│100  │-122.0839,37.3861│1742232589    │tyler │[0.1, 0.4, 0.5] │high          │
└────────┴─────┴─────────────────┴──────────────┴──────┴────────────────┴──────────────┘


## Cleanup

In [17]:
// Clean up JSON index
jindex.delete(true);
System.out.println("JSON index deleted");

JSON index deleted


# Working with nested data in JSON

Redis also supports native **JSON** objects. These can be multi-level (nested) objects, with full JSONPath support for updating/retrieving sub elements:

```json
{
    "name": "Specialized Stump jumper",
    "metadata": {
        "model": "Stumpjumper",
        "brand": "Specialized",
        "type": "Enduro bikes",
        "price": 3000
    },
}
```

#### Full JSON Path support
Because Redis enables full JSON path support, when creating an index schema, elements need to be indexed and selected by their path with the desired `name` AND `path` that points to where the data is located within the objects.

> By default, RedisVL4J will assume the path as `$.{name}` if not provided in JSON fields schema. If nested provide path as `$.object.attribute`

### As an example:

In [18]:
// Create bike data with nested metadata
// Descriptions
var desc1 = "The Specialized Stumpjumper is a versatile enduro bike that dominates both climbs and descents. "
  + "Features a FACT 11m carbon fiber frame, FOX FLOAT suspension with 160mm travel, and SRAM X01 Eagle drivetrain. "
  + "The asymmetric frame design and internal storage compartment make it a practical choice for all-day adventures.";

var desc2 = "Trek's Slash is built for aggressive enduro riding and racing. "
  + "Featuring Trek's Alpha Aluminum frame with RE:aktiv suspension technology, 160mm travel, and Knock Block frame protection. "
  + "Equipped with Bontrager components and a Shimano XT drivetrain, this bike excels on technical trails and enduro race courses.";

// Real embeddings
float[] embeddingArray1 = embeddingModel.embed(desc1).content().vector();
float[] embeddingArray2 = embeddingModel.embed(desc2).content().vector();

var bikeData = List.<Map<String, Object>>of(
  Map.<String, Object>of(
    "name", "Specialized Stumpjumper",
    "metadata", Map.of(
      "model", "Stumpjumper",
      "brand", "Specialized",
      "type", "Enduro bikes",
      "price", 3000
    ),
    "description", desc1,
    "bike_embedding", embeddingArray1
  ),
  Map.<String, Object>of(
    "name", "bike_2",
    "metadata", Map.of(
      "model", "Slash",
      "brand", "Trek",
      "type", "Enduro bikes",
      "price", 5000
    ),
    "description", desc2,
    "bike_embedding", embeddingArray2
  )
);

// Optional
System.out.println("Loaded " + bikeData.size() + " bikes");


// Define bike schema with nested JSON paths
Map<String, Object> bikeSchema = Map.of(
    "index", Map.of(
        "name", "bike-json",
        "prefix", "bike-json",
        "storage_type", "json"  // JSON storage type
    ),
    "fields", List.of(
        Map.of(
            "name", "model",
            "type", "tag",
            "path", "$.metadata.model"  // note the '$'
        ),
        Map.of(
            "name", "brand",
            "type", "tag",
            "path", "$.metadata.brand"
        ),
        Map.of(
            "name", "price",
            "type", "numeric",
            "path", "$.metadata.price"
        ),
        Map.of(
            "name", "bike_embedding",
            "type", "vector",
            "attrs", Map.of(
                "dims", embeddingArray1.length,  // Use actual embedding dimensions
                "distance_metric", "cosine",
                "algorithm", "flat",
                "datatype", "float32"
            )
            // No explicit path - should default to $.bike_embedding
        )
    )
);

System.out.println("Created bike data with nested metadata and " + embeddingArray1.length + "-dim embeddings");
System.out.println("Embeddings generated using AllMiniLmL6V2 model");

Loaded 2 bikes
Created bike data with nested metadata and 384-dim embeddings
Embeddings generated using AllMiniLmL6V2 model


In [19]:
// Construct a search index from the json schema
SearchIndex bikeIndex = SearchIndex.fromDict(bikeSchema, jedis);

// Create the index (no data yet)
bikeIndex.create(true);

System.out.println("Bike index created: " + bikeIndex.getName());

Bike index created: bike-json


In [20]:
// Load bike data
List<String> bikeKeys = bikeIndex.load(bikeData);

System.out.println("Loaded " + bikeKeys.size() + " bikes");
bikeKeys.forEach(key -> System.out.println("  " + key));

Loaded 2 bikes
  bike-json:01K379F22GC6723VVGD6KT295B
  bike-json:01K379F22KE06T84THDNJQBT54


In [21]:
// Create a semantic search query
String queryText = "I'd like a bike for aggressive riding";

// Generate embedding for the query using the same model - use .content() to get the Embedding
float[] queryVector = embeddingModel.embed(queryText).content().vector();

VectorQuery bikeQuery = VectorQuery.builder()
    .vector(queryVector)
    .field("$.bike_embedding")  // Use JSON path for JSON storage
    .returnFields(
        "$.metadata.brand",  // Use JSON path for indexed nested field
        "$.name",
        "$.metadata.type"  // Note: retrieving non-indexed field requires full path
    )
    .numResults(2)
    .build();

List<Map<String, Object>> bikeResults = bikeIndex.query(bikeQuery);

System.out.println("\nSemantic search for: '" + queryText + "'");
System.out.println("Results found: " + bikeResults.size());

DEBUG: Executing search query: '*=>[KNN $K @\$\.bike_embedding $vec AS vector_distance]'
DEBUG: Search params: {vec=[B@7525b9b, K=2}

Semantic search for: 'I'd like a bike for aggressive riding'
Results found: 2


**Note:** As shown in the example if you want to retrieve a field from json object that was not indexed you will also need to supply the full path as with `$.metadata.type`.

In [22]:
// Display the results with similarity scores
bikeResults.forEach(result -> {
    String id = (String) result.get("id");

    // For JSON storage, results come back with JSON paths
    // Handle both nested objects and direct values
    Object metadataObj = result.get("$.metadata");
    String brand = null;
    String type = null;

    if (metadataObj instanceof Map) {
        Map<String, Object> metadata = (Map<String, Object>) metadataObj;
        brand = (String) metadata.get("brand");
        type = (String) metadata.get("type");
    }

    String name = (String) result.get("$.name");
    String distance = (String) result.get("vector_distance");

    // Convert distance to similarity score (1 - cosine distance)
    double similarity = 1.0 - Double.parseDouble(distance);

    System.out.println("ID: " + id);
    System.out.println("  Brand: " + brand);
    System.out.println("  Name: " + name);
    System.out.println("  Type: " + type);
    System.out.println("  Cosine Distance: " + distance);
    System.out.println("  Similarity Score: " + String.format("%.4f", similarity));
    System.out.println();
});

// The bike with the lower distance (higher similarity) should be Trek Slash
// since it's described as "aggressive enduro riding and racing"

ID: bike-json:01K379F22GC6723VVGD6KT295B
  Brand: Specialized
  Name: Specialized Stumpjumper
  Type: Enduro bikes
  Cosine Distance: 0.623969972134
  Similarity Score: 0.3760

ID: bike-json:01K379F22KE06T84THDNJQBT54
  Brand: Trek
  Name: bike_2
  Type: Enduro bikes
  Cosine Distance: 0.579276919365
  Similarity Score: 0.4207



In [23]:
// Let's try another query to see the model understands different contexts
String query2 = "versatile bike for all-day trail adventures";

// Use .content() to get the Embedding
float[] queryVector2 = embeddingModel.embed(query2).content().vector();

VectorQuery bikeQuery2 = VectorQuery.builder()
    .vector(queryVector2)
    .field("$.bike_embedding")  // Use JSON path
    .returnFields("$.metadata.brand", "$.name")
    .numResults(2)
    .build();

List<Map<String, Object>> results2 = bikeIndex.query(bikeQuery2);

System.out.println("\nSemantic search for: '" + query2 + "'");
results2.forEach(result -> {
    // Handle nested metadata structure
    Object metadataObj = result.get("$.metadata");
    String brand = null;
    if (metadataObj instanceof Map) {
        Map<String, Object> metadata = (Map<String, Object>) metadataObj;
        brand = (String) metadata.get("brand");
    }

    String name = (String) result.get("$.name");
    String distance = (String) result.get("vector_distance");
    double similarity = 1.0 - Double.parseDouble(distance);

    System.out.println(brand + " - " + name + " (similarity: " + String.format("%.4f", similarity) + ")");
});

// This query should rank the Specialized Stumpjumper higher
// since it's described as "versatile" and "practical choice for all-day adventures"

DEBUG: Executing search query: '*=>[KNN $K @\$\.bike_embedding $vec AS vector_distance]'
DEBUG: Search params: {vec=[B@176dfae0, K=2}

Semantic search for: 'versatile bike for all-day trail adventures'
Specialized - Specialized Stumpjumper (similarity: 0.4292)
Trek - bike_2 (similarity: 0.3124)


# Cleanup

In [24]:
// Clean up bike index
bikeIndex.delete(true);
System.out.println("Bike index deleted");

// Close Redis connection
jedis.close();
System.out.println("Redis connection closed");

Bike index deleted
Redis connection closed
